In [5]:
import numpy as np

## 熱・水分の収支計算をするクラス  

In [12]:
class Calculation():

    def __init__( self, cell, dt ):
        self.cell     = cell
        self.dt       = dt
         
    # 熱・水分収支計算
    def differenceOfFlux( self, flux_mns, flux_pls ):
        self.flux_mns = flux_mns
        self.flux_pls = flux_pls
    
    def newTemp( self ):
        self.W = self.flux_mns.jv - self.flux_pls.jv
        self.temp = cal_newTemp( crow = self.cell.crow, temp = self.cell.temp, dq = ( self.flux_mns.qs - self.flux_pls.qs ),\
                                dx = self.cell.dx, W = self.W, dt = self.dt )
    def newTemp_NoMoisture( self ):
        self.temp = cal_newTemp_noMoisture( crow = self.cell.crow, temp = self.cell.temp, dq = ( self.flux_mns.qs - self.flux_pls.qs ),\
                                           dx = self.cell.dx, dt = self.dt )
        
    def newMiu( self ):
        self.djw = ( self.flux_mns.jv + self.flux_mns.jl ) - ( self.flux_pls.jv + self.flux_pls.jl )
        self.miu = cal_newMiu( dphi = self.cell.dphi, miu = self.cell.miu, djw = self.djw, dx = self.cell.dx, dt = self.dt )
        
    # 境界セルの計算
    # 熱収支
    def boundaryCell_temp( self, boundaryFlux, boundary ):
        if boundary.heatcondition  == 'dirichlit':
            self.temp = boundary.temp
        elif boundary.heatcondition == 'neumann':
            self.dqs  = neumann_Heat( self.cell, boundaryFlux, boundary.qs )
            self.temp = cal_newTemp( self.cell.crow, self.cell.temp, self.dqs, self.cell.dx, self.djv, self.dt )
        elif boundary.heatcondition ==  'robin':
            self.dqs  = robin_Heat_Air( AL = boundary.al, tempOut = boundary.temp, cell = self.cell, flux = boundaryFlux )
            self.temp = cal_newTemp( crow = self.cell.crow, temp = self.cell.temp, dq = self.dqs,\
                                    dx = self.cell.dx, W = self.djv, dt = self.dt )  # 水の収支から必ず先に解くべし
        else :
            raise ValueError("境界条件クラスboundaryのsetHeatConditionにはdirichlit, neumann,robinのいずれかを入力してください" )
            
    # 水分収支
    def boundaryCell_Miu( self, boundaryFlux, boundary ):
        if   boundary.vapourcondition == 'dirichlit':
            self.miu = boundary.miu
        elif boundary.vapourcondition == 'neumann':
            self.djv  = neumann_Vapour( self.cell, boundaryFlux, boundary.jv )
        elif boundary.vapourcondition == 'robin':
            self.djv  = robin_Vapour_Air( boundary.aldp, boundary.pv, self.cell, boundaryFlux )
        else :
            raise ValueError("境界条件クラスboundaryのsetVapourConditionにはdirichlit, neumann,robinのいずれかを入力してください" )
            
        if   boundary.liquidcondition == 'dirichlit':
            self.miu = boundary.miu
        elif boundary.liquidcondition == 'neumann':
            self.djl  = neumann_Liquid( self.cell, boundaryFlux, boundary.jl )
        else :
            raise ValueError("境界条件クラスboundaryのsetVapourConditionにはdirichlit, neumann,robinのいずれかを入力してください" )
        
        self.miu = cal_newMiu( dphi = self.cell.dphi, miu = self.cell.miu, djw = self.djv + self.djl, dx = self.cell.dx, dt = self.dt )
    

In [ ]:
c = Calculation(0.01, 0.1 )

### 熱収支式  
#### 水分収支無し  

基礎方程式：$c\rho\frac{\partial T}{\partial t} = -\nabla・q$  

時間差分解：$T_{t+1} = T_{t} - \frac{q(x)-q(x+1)}{dx}\frac{dt}{c\rho}$  

In [ ]:
def cal_newTemp_noMoisture( crow, temp, dq, dx, dt ):
    return temp + dq / dx * ( dt / crow )

#### 水分収支あり  
基礎方程式：$c\rho\frac{\partial T}{\partial t} = -\nabla・q + rW$  

時間差分解：$T_{t+1} = T_{t} - \frac{(q(x)-q(x+1) - rW )}{dx}\frac{dt}{c\rho}$  


In [6]:
def cal_newTemp( crow, temp, dq, W, dx, dt ):
    r = ratentHeat( temp )
    return temp + ( dq + r * W ) / dx * ( dt / crow )

In [5]:
def ratentHeat( temp ):
    return ( 597.5 - 0.559 * ( temp - 273.16 ) ) * 4186.05


### 水分収支式  
基礎方程式：$\frac{\partial[(\phi_{0}-\psi)\rho_{v}+\rho_{w}\psi]}{\partial t} = -\nabla・J_w$  

$\phi_{0}$：絶乾時の材料の空隙率[-]  
$\psi$：含水率[-]  
$\phi_{0}-\psi$：含水率$\psi$の時の空隙率[-]  
$\rho_w, \rho_v$：それぞれ液水、水蒸気の密度[kg/m3]  
$J_w$：液水の流量

#### 水分化学ポテンシャルベース  

$\rho_{w}\frac{\partial\psi}{\partial \mu}\frac{\partial\mu}{\partial t} = -\nabla・J_w$  

時間差分解：$\mu_{t+1} = \mu_{t} - \frac{J_w(x) -J_w(x+1)}{dx} \frac{\partial\mu}{\partial \psi} \frac{dt}{\rho_{w}}$  

In [7]:
def cal_newMiu( dphi, miu, djw, dx, dt ):
    roww = 1000.0
    return miu + djw / dx / dphi * ( dt / roww )


#### 含水率ベース（高含水域）  
$\rho_w >> \rho_v,　\rho_w \approx const$、より高含水域では、水分の収支式を以下のように近似できる。  

$\rho_{w}\frac{\partial\psi}{\partial t} = -\nabla・J_w$  

時間差分解：$\phi_{t+1} = \phi_{t} - \frac{J_w(x) -J_w(x+1)}{dx} \frac{dt}{\rho_{w}}$

In [ ]:
def cal_newPhi( phi, djw, dx, dt ):
    roww = 1000.0
    return phi + djw / dx * ( dt / roww )


#### ハイグロスコピック（蒸気拡散支配領域）    

$(\phi_0\gamma'+\kappa) \frac{\partial X}{\partial t} - \nu \frac{\partial T}{\partial t} = \lambda'_x\frac{\partial^2 X}{\partial x^2}$  

$- r\kappa \frac{\partial X}{\partial t}  + (c\rho +r\nu) \frac{\partial T}{\partial t} = \lambda \frac{\partial^2 T}{\partial x^2}$  

ただし  
$\kappa = \rho_w (\frac{\partial \psi}{\partial X})_T$,　  $\nu = \rho_w(\frac{\partial \psi}{\partial T})_X$  

$\gamma'$：乾燥空気の密度[kg/m3]

In [1]:
# 未実装

### 境界条件を計算する関数群

### 境界条件のクラス  
cellクラスと同様に構造体として値を保持するだけのクラス  
境界の温度・湿度情報、境界の流量情報を保持する  
Dirichlet境界条件  
Neumann境界条件  
Robin境界条件  
に対応する（2018/07/18時点）  
それぞれheatcondition, vapourcondition, liquidconditionには  
"dirichlet", "neumann", "robin"のいずれかを入力すること

In [3]:
class BoundaryCondition():
    
    def __init__( self, position ):
        self.position = position
        
    ######################################
    # 境界条件情報
    # 熱境界情報
    def setHeatCondition( self, initial ):
        self.heatcondition = initial
    def setVapourCondition( self, initial ):
        self.vapourcondition = initial
    def setLiquidCondition( self, initial ):
        self.liquidcondition = initial
     
    ######################################
    # 温度情報
    def setTemp( self, initial = 0.0 ):
        self.temp = initial
        
    ######################################
    # 水蒸気の情報
    def setRH( self, initial = 0.0  ):
        self.rh = initial
    def setMiu( self, initial = 0.0 ):
        self.miu = initial 
    def setPv( self, initial = 0.0 ):
        self.pv = initial
        
    ######################################
    # 流量の情報
    def setQs( self, initial = 0.0 ):
        self.qs = initial
    def setJv( self, initial = 0.0 ):
        self.jv = initial 
    def setJl( self, initial = 0.0 ):
        self.jl = initial 
        
    ######################################
    # 伝達率
    def setAL( self, initial = 0.0 ):
        self.al = initial
    def setALDP( self, initial = 0.0 ):
        self.aldp = initial
        

#### 熱伝達  
定義：$\dot q = -\alpha \Delta T$    
$\alpha$：熱伝達率[W/m2K]

In [4]:
def heatConvection( AL= 0.0, dtemp= 0.0 ):
    return AL * dtemp

#### 水分伝達（水蒸気圧勾配）  
定義：$J_v = -\alpha^{'}_{p}\Delta P_v$  
$J_v$：気相水分流量[kg/m2s]  
$\alpha^{'}_{p}$：水分伝達率[kg/m2sPa]  
$P_v$：水蒸気圧[Pa]

In [5]:
def vapourConvection_pressure( ALDP= 0.0, dpv= 0.0 ):
    return ALDP * dpv

#### 第一種境界条件  
端のセルにある温度・水分状態を与える

#### 第二境界条件
端のセルにある流量を与える

In [8]:
def neumann_Heat( cell, flux, qs, dt ):
    if cell.position == 0:
        dq = qs - flux.qs
    else :
        dq = flux.qs - qs
    return cal_newTemp( cell.crow, cell.temp, dq, cell.dx, dt )

def neumann_Liquid( cell, flux, jl ):
    if cell.position == 0:
        djl = jl - flux.jl
    else :
        djl = flux.jl - jl
    return djl

def neumann_Vapour( cell, flux, jv ):
    if cell.position == 0:
        djv = jv - flux.jv
    else :
        djv = flux.jv - jv
    return djv


#### 第三種境界条件  
端に隣接するセルにある温度・水分状態を与えることで流量計算を行う  

関数：robin_Heat_Air( AL, tempOut, cell, flux, qs, dt )  
…空気との熱伝達を行う場合  

関数：robin_Water_Air( ALD, pvOut, cell, flux, jv, jl, dt )  
…空気との水分伝達を行う場合  

※ 一定塩濃度の塩溶液と接する...といったような場合は第一種境界条件で良い

In [10]:
def robin_Heat_Air( AL, tempOut, cell, flux ):
    if cell.position == 0:
        dtemp = tempOut - cell.temp
        qb = heatConvection( AL, dtemp )
        dq = qb - flux.qs
    else:
        dtemp = cell.temp - tempOut
        qb = heatConvection( AL, dtemp )
        dq = flux.qs - qb
    return dq

def robin_Vapour_Air( ALDP, pvOut, cell, flux ):
    if cell.position == 0:
        dpv = pvOut - cell.pv
        jvb = vapourConvection_pressure( ALDP, dpv )
        djv = jvb - flux.jv
    else:
        dpv = cell.pv - pvOut 
        jvb = vapourConvection_pressure( ALDP, dpv )
        djv = flux.jv - jvb
    return djv